In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 14993228
paper_name = 'serrano_arino_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/serrano_arino_2004.xlsx', sheet_name='Sheet1', header=None)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 118 x 2


In [10]:
original_data.head()

,0,1
0,CUP5,1
1,TFP1,1
2,VMA2,1
3,VMA22,1
4,VMA4,1


In [11]:
original_data['gene'] = original_data[0].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, gene, orf]
Index: []


In [15]:
# Flip the scores such that -5 is the most sensitive and -1 is the least sensitive
original_data['data'] = pd.to_numeric(original_data[1], errors='coerce') - 6

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(118, 1)

# Load & process tested strains

In [20]:
tested = pd.read_excel('raw_data/BY4741.xlsx', sheet_name='Tabelle1')

In [21]:
tested.head()

,record #,ORF,strain,batch,row,col,comments,slow growth?
0,338,YAL068C,BY4741,chr1_1,A,2,NaN,NaN
1,339,YAL067C,BY4741,chr1_1,A,3,NaN,NaN
2,340,YAL066W,BY4741,chr1_1,A,4,NaN,NaN
3,341,YAL065C,BY4741,chr1_1,A,5,NaN,NaN
4,345,YAL062W,BY4741,chr1_1,A,9,NaN,NaN


In [22]:
tested['orf'] = tested['ORF'].astype(str)

In [23]:
tested['orf'] = clean_orf(tested['orf'])

In [24]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [25]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [record #, ORF, strain, batch, row, col, comments, slow growth?, orf]
Index: []


In [26]:
tested_orfs = tested['orf'].unique()

In [27]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YBR011C', 'YBR132C', 'YCL005W-A', 'YMR038C']

In [28]:
tested_orfs = list(tested_orfs) + missing

In [29]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [30]:
data = original_data.copy()

In [31]:
dataset_ids = [187]
datasets = datasets.reindex(index=dataset_ids)

In [32]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [33]:
data.head()

dataset_id,187
data_type,value
orf,
YAL068C,0
YAL067C,0
YAL066W,0
YAL065C,0
YAL062W,0


## Subset to the genes currently in SGD

In [34]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [35]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,187
,data_type,value
gene_id,orf,
1869,YAL068C,0
61,YAL067C,0
60,YAL066W,0
1727,YAL065C,0
57,YAL062W,0


# Normalize

In [36]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [37]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [38]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        187       
data_type       value valuez
gene_id orf                 
1869    YAL068C     0    0.0
61      YAL067C     0    0.0
60      YAL066W     0    0.0
1727    YAL065C     0    0.0
57      YAL062W     0    0.0

# Print out

In [39]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [40]:
from IO.save_data_to_db3 import *

In [41]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 14993228...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.02s/it]

Updating the data_modified_on field...
